In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.utils import resample
from linearmodels.panel import RandomEffects 

# Load the datasets
nlswork_df = pd.read_stata("/Users/danielseymour/Downloads/nlswork.dta")
guns_df = pd.read_stata("/Users/danielseymour/Downloads/Guns.dta")

# display(nlswork_df.head())
# nlswork_df.info()

## What is the shall gun issue?
The "shall-issue" policy refers to state laws governing concealed carry permits for firearms. Under shall-issue laws, authorities must issue a concealed carry permit to any applicant who meets basic requirements (like age, background check, training), without requiring them to demonstrate a specific need.
This contrasts with "may-issue" states, where authorities have discretion to deny permits even if basic requirements are met. As of 2024, most U.S. states have shall-issue policies or permitless carry, with only a few maintaining may-issue systems.

In [2]:
guns_df.head()

,year,vio,mur,rob,incarc_rate,pb1064,pw1064,pm1029,pop,avginc,density,stateid,shall
0,77,414.399994,14.2,96.800003,83,8.384873,55.122906,18.174412,3.780403,9.563148,0.074552,1,0
1,78,419.100006,13.3,99.099998,94,8.352101,55.143665,17.994080,3.831838,9.932000,0.075567,1,0
2,79,413.299988,13.2,109.500000,144,8.329575,55.135857,17.839336,3.866248,9.877028,0.076245,1,0
3,80,448.500000,13.2,132.100006,141,8.408386,54.912586,17.734198,3.900368,9.541428,0.076829,1,0
4,81,470.500000,11.9,126.500000,149,8.483435,54.925125,17.673716,3.918531,9.548351,0.077187,1,0


In [3]:
guns_df.describe()

,year,vio,mur,rob,incarc_rate,pb1064,pw1064,pm1029,pop,avginc,density,stateid,shall
count,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000,1173.000000
mean,88.000000,503.074707,7.665132,161.820206,226.579710,5.336217,62.945431,16.081129,4.816341,13.724796,0.352038,28.960784,0.242967
std,6.636079,334.277039,7.522707,170.509842,178.888094,4.885689,9.761533,1.732143,5.252115,2.554542,1.355472,15.683522,0.429058
min,77.000000,47.000000,0.200000,6.400000,19.000000,0.248207,21.780430,12.213680,0.402753,8.554884,0.000707,1.000000,0.000000
25%,82.000000,283.100006,3.700000,71.099998,114.000000,2.202196,59.939705,14.653370,1.187706,11.934755,0.031911,16.000000,0.000000
50%,88.000000,443.000000,6.400000,124.099998,187.000000,4.026213,65.061279,15.895169,3.271332,13.401551,0.081569,29.000000,0.000000
75%,94.000000,650.900024,9.800000,192.699997,291.000000,6.850673,69.200096,17.525715,5.685611,15.271010,0.177718,42.000000,0.000000
max,99.000000,2921.800049,80.599998,1635.099976,1913.000000,26.979570,76.525749,22.352686,33.145123,23.646713,11.102116,56.000000,1.000000


In [4]:
guns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         1173 non-null   int8   
 1   vio          1173 non-null   float32
 2   mur          1173 non-null   float32
 3   rob          1173 non-null   float32
 4   incarc_rate  1173 non-null   int16  
 5   pb1064       1173 non-null   float32
 6   pw1064       1173 non-null   float32
 7   pm1029       1173 non-null   float32
 8   pop          1173 non-null   float32
 9   avginc       1173 non-null   float32
 10  density      1173 non-null   float32
 11  stateid      1173 non-null   int8   
 12  shall        1173 non-null   int8   
dtypes: float32(9), int16(1), int8(3)
memory usage: 47.1 KB


In [6]:
# Run the regression with state and year fixed effects
# Remember the variables rob and mur are bad controls as they are outcomes of the violence variable

# Fixed Effects with Year Dummy Variables
model_fe = smf.ols("np.log(vio) ~ shall + pb1064 + pw1064 + pm1029 + incarc_rate + pop + avginc + density + C(year) + C(stateid)", 
                    data=guns_df).fit(cov_type='cluster', cov_kwds={'groups': guns_df['stateid']})

# Display regression results
print(model_fe.summary())

                            OLS Regression Results                            
Dep. Variable:            np.log(vio)   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     9593.
Date:                Sun, 23 Feb 2025   Prob (F-statistic):           1.49e-84
Time:                        17:13:02   Log-Likelihood:                 683.58
No. Observations:                1173   AIC:                            -1205.
Df Residuals:                    1092   BIC:                            -794.7
Df Model:                          80                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            3.9720      1.110  

/Applications/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 80, but rank is 30
  warnings.warn('covariance of constraints does not have full '


In [7]:
# To check if state fixed effects were run

print(model_fe.df_model)  # Number of estimated coefficients
print(model_fe.df_resid)  # Residual degrees of freedom

80.0
1092.0


In [8]:
print(model_fe.model.exog_names)  # List of variables in the regression

['Intercept', 'C(year)[T.78]', 'C(year)[T.79]', 'C(year)[T.80]', 'C(year)[T.81]', 'C(year)[T.82]', 'C(year)[T.83]', 'C(year)[T.84]', 'C(year)[T.85]', 'C(year)[T.86]', 'C(year)[T.87]', 'C(year)[T.88]', 'C(year)[T.89]', 'C(year)[T.90]', 'C(year)[T.91]', 'C(year)[T.92]', 'C(year)[T.93]', 'C(year)[T.94]', 'C(year)[T.95]', 'C(year)[T.96]', 'C(year)[T.97]', 'C(year)[T.98]', 'C(year)[T.99]', 'C(stateid)[T.2]', 'C(stateid)[T.4]', 'C(stateid)[T.5]', 'C(stateid)[T.6]', 'C(stateid)[T.8]', 'C(stateid)[T.9]', 'C(stateid)[T.10]', 'C(stateid)[T.11]', 'C(stateid)[T.12]', 'C(stateid)[T.13]', 'C(stateid)[T.15]', 'C(stateid)[T.16]', 'C(stateid)[T.17]', 'C(stateid)[T.18]', 'C(stateid)[T.19]', 'C(stateid)[T.20]', 'C(stateid)[T.21]', 'C(stateid)[T.22]', 'C(stateid)[T.23]', 'C(stateid)[T.24]', 'C(stateid)[T.25]', 'C(stateid)[T.26]', 'C(stateid)[T.27]', 'C(stateid)[T.28]', 'C(stateid)[T.29]', 'C(stateid)[T.30]', 'C(stateid)[T.31]', 'C(stateid)[T.32]', 'C(stateid)[T.33]', 'C(stateid)[T.34]', 'C(stateid)[T.35]'

## Interpretation
Holding all else constant (year, state, and other factors), states with “shall issue” laws are associated with an average 28.78% decrease in violent crime compared to states without such laws.

Recall:
How to Correctly Interpret the Coefficient?

I think there's a trick which involves taking the derivative!

The coefficient on shall is -0.2878, which means:

$$\frac{\partial \ln(Y)}{\partial X} = \beta$$

Since  \ln(Y)  is the dependent variable, the estimated coefficient represents the proportional change in  Y  (violence) when shall increases by one unit (i.e., “shall issue” law is in effect).

To convert this into a percentage change, use the approximation:

$$\% \Delta Y = 100 \times \beta$$


Thus, for shall:

$$\% \Delta \text{vio} = 100 \times (-0.2878) = -28.78\%$$





# PS5

Use the data on ‘shall issue’ laws in the 50 US states and DC for 23 years, used
in a previous problem set. Let us proceed with the ”within” estimator for fixed
eﬀects with both state and year eﬀects and now consider the question of which
standard errors to use. Use ‘ln(vio)’ as dependent variable.
(1) Compute the canonical White standard errors for this model and report
the results.

In [9]:
# model = sm.OLS(y, X).fit(cov_type='HC0')
# In statsmodels, you can compute White’s heteroskedasticity-consistent standard errors (also called HC0 in the literature) 
# by specifying the cov_type='HC0' argument when fitting the model. 

model_fe_white = smf.ols("np.log(vio) ~ shall + pb1064 + pw1064 + pm1029 + incarc_rate + pop + avginc + density + C(year) + C(stateid)", 
                    data=guns_df).fit(cov_type='HC0')

print(model_fe_white.summary())

                            OLS Regression Results                            
Dep. Variable:            np.log(vio)   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     515.4
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        17:13:06   Log-Likelihood:                 683.58
No. Observations:                1173   AIC:                            -1205.
Df Residuals:                    1092   BIC:                            -794.7
Df Model:                          80                                         
Covariance Type:                  HC0                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            3.9720      0.433  

## Clustered se: 0.124

## White heteroskedaskicty-robust se: 0.019

The clustered standard errors are an order of magnitude larger. Why is this? Does it mean that the White heteroskedaskicty-robust standard errors aren't including a source of sampling error that clustered standard errors does? YES.

White (heteroskedasticity-robust) standard errors assume that each observation is independent, but that the variance of the error term may differ from observation to observation (heteroskedasticity). They do not account for the possibility that observations within the same “cluster” may share correlated error terms.

Cluster-robust standard errors, on the other hand, do two things simultaneously:
	1.	Allow for heteroskedasticity, and
	2.	Allow for arbitrary correlation of the error terms within each cluster (e.g., state, county, or individual if you have panel data).
    
When there is meaningful intra-cluster correlation — i.e., observations within the same cluster are “more alike” than those in different clusters — White’s robust standard errors will tend to be too small. That is because White’s robust approach is still treating each observation as though it provides unique, independent information.



# Calculate cluster (block) bootstrap standard errors

In [10]:
# Define the number of bootstrap samples
n_bootstraps = 1000

# Define clusters (states)
clusters = guns_df['stateid'].unique()

# Store bootstrap coefficients
bootstrap_coefs = []

# Run Bootstrap Procedure
for _ in range(n_bootstraps):
    # Randomly select a subset of states (clusters) with replacement from the unique set of states.
    sampled_states = resample(clusters, replace=True)
    
    # Select all observations from resampled states
    bootstrap_sample = guns_df[guns_df['stateid'].isin(sampled_states)]
    
    # Run fixed effects regression
    model = smf.ols("np.log(vio) ~ shall + pb1064 + pw1064 + pm1029 + incarc_rate + pop + avginc + density + C(year) + C(stateid)", 
                    data=bootstrap_sample).fit()
    
    # Store coefficient of interest (shall issue)
    bootstrap_coefs.append(model.params['shall'])

# Compute Bootstrap Standard Error
bootstrap_se = np.std(bootstrap_coefs)

# Display results
print(f"Bootstrap Clustered Standard Error for 'shall': {bootstrap_se:.4f}")

Bootstrap Clustered Standard Error for 'shall': 0.0338


## Estimate a random eﬀect (RE) model with clustered standard errors at the state level
(5) Use delta method to compute the standard error for the predicted values
at means

(6) Use delta method to compute the standard error for the predicted values
at means in level

A 2-level MultiIndex (also called a hierarchical index) in pandas is an indexing scheme for a DataFrame or Series where each row (or entry) is uniquely identified by two levels of keys rather than just one. In the context of panel data, this typically means you have an “entity” (first level) and a “time” (second level) that together identify each row.

In [11]:
# Estimate baseline RE model

# Define independent and dependent variables
X = guns_df[['shall', 'pb1064', 'pw1064', 'pm1029', 'incarc_rate', 'pop', 'avginc', 'density']]
y = np.log(guns_df['vio'])  # log transformation of violence

# Add constant (intercept)
X = sm.add_constant(X)

# Estimate Random Effects model
model_re = RandomEffects(y, X).fit(cov_type='clustered', cluster_entity=True)

# Display results
print(model_re.summary)

ValueError: Series can only be used with a 2-level MultiIndex